In [1]:
from sympy import *

# Formulações algébricas para determinação de forças de reação sob uma barra
## Considerações iniciais
Considera-se uma barra determinada por um segmento de reta. A barra sofre um momento fletor resultante $M_R$ e um vetor força resultante $\vec{F_R}$. A partir disto, se deseja detertminar duas forças de reação $\vec{F_A}$ e $\vec{F_B}$, com excentricidades de $\vec{R_A}$ e $\vec{R_B}$ em relação a um ponto de referência arbitrário.

Para que seja um sistema isostático com duas forças de reação, uma delas deve está restrita a uma direção específica $\hat{i}$ de modo que $\vec{F_B} = k \hat{i}$, onde $k$ representa uma das icógnitas do problema.

Sabemos também que, pelo princípio da inércia: $\vec{F_R} = \vec{F_A} + \vec{F_B}$. Fazendo a mesma consideração para os momentos fletores das forças, chegamos no primeiro sistema de equações:

$\begin{cases}
\vec{F_R} = \vec{F_A} + k \hat{i} \\
M_R = M_A + M_R
\end{cases}$

Para simplificar a manipulação algébrica, foi cosiderado que $\vec{F_R} = \langle C_x; C_y \rangle$, $\vec{F_B} = \langle B_x; B_y \rangle$,  $\vec{F_A} = \langle A_x; A_y \rangle$, $\hat{i} = \langle i_x; i_y \rangle$, $\vec{R_A} = \langle R_{xA}; R_{yA} \rangle$ e, por fim, $\vec{R_B} = \langle R_{xB}; R_{yB} \rangle$.

Começamos definindo os símbolos no `sympy`.

In [2]:
ax, ay = symbols('A_x A_y')
bx, by = symbols('B_x B_y')
cx, cy = symbols('C_x C_y')
ix, iy = symbols('i_x i_y')
rxa, rya = symbols('R_xA R_yA')
rxb, ryb = symbols('R_xB R_yB')

k, mr = symbols('k M_R')

Em seguida, definimos a equação de estaticidade do momento fletor, onde já é feito o produto cruzado de $M = \vec{R} \times \vec {F} = R_x F_y - R_y F_x$. E também a equação de estacidade de forças, dividindo a equação nas duas componentes. Mas reescrevemos $B_x$ em termos de $k$ e $\hat{i}$. 

In [3]:
bx = k * ix
by = k * iy

ma = rxa * ay - rya * ax
mb = rxb * by - ryb * bx
exp1 = Eq(mr, ma + mb)

exp1

Eq(M_R, -A_x*R_yA + A_y*R_xA + R_xB*i_y*k - R_yB*i_x*k)

In [4]:
exp2 = Eq(cx, ax + k * ix)

exp2

Eq(C_x, A_x + i_x*k)

In [5]:
exp3 = Eq(cy, ay + k * iy)

exp3

Eq(C_y, A_y + i_y*k)

## Determinando $k$

In [6]:
expk = Eq(k, solve(exp1, k)[0])
expk

Eq(k, (A_x*R_yA - A_y*R_xA + M_R)/(R_xB*i_y - R_yB*i_x))

Vamos subsitutir o denominador, que é constante, por $\phi$.

In [7]:
phi = symbols('\phi')

exp_phi = rxb * iy - ryb * ix

expk = expk.subs(exp_phi, phi)

expk

Eq(k, (A_x*R_yA - A_y*R_xA + M_R)/\phi)

In [8]:
expk = expk.args[1]

Agora substituimos isto na expressão 2 e na 3. 

In [9]:
exp2 = exp2.subs(k, expk)
exp3 = exp3.subs(k, expk)

## Resolvendo o sistema de equações
Com as duas expressões encontradas, resolver um sistema com as duas expressões.

In [10]:
linsolve([exp2, exp3], ax, ay)

FiniteSet(((C_x*R_xA*i_y - C_x*\phi - C_y*R_xA*i_x + M_R*i_x)/(R_xA*i_y - R_yA*i_x - \phi), (C_x*R_yA*i_y - C_y*R_yA*i_x - C_y*\phi + M_R*i_y)/(R_xA*i_y - R_yA*i_x - \phi)))

In [11]:
expAx = linsolve([exp2, exp3], ax, ay).args[0].args[0]
expAy = linsolve([exp2, exp3], ax, ay).args[0].args[1]

Para simplificar ainda mais, podemos substituir $|\vec{R_A}\times \hat{i}|$ por $\psi$.

In [12]:
psi = Symbol('\psi')

exp_psi = rxa * iy - rya * ix

exp_psi

R_xA*i_y - R_yA*i_x

In [13]:
expAxs = expAx.subs(exp_psi, psi)
expAys = expAy.subs(exp_psi, psi)

In [14]:
expAxs

(C_x*R_xA*i_y - C_x*\phi - C_y*R_xA*i_x + M_R*i_x)/(-\phi + \psi)

In [15]:
expAys

(C_x*R_yA*i_y - C_y*R_yA*i_x - C_y*\phi + M_R*i_y)/(-\phi + \psi)

## Validação prática

Vamos testar as fórmulas com um modelo simples e previsível, uma barra apoiada nos pontos $(0;2)$ e $(3;2)$, de modo que estes valem repectivamente aos vetores $\vec{R_A}$ e $\vec{R_B}$. A unica carga na barra é uma força determinada como $\vec{F_R} = \langle 0; 3 \rangle$, de modo que $M_R = 3$, além disso, $\hat{i} = \langle 0; 1 \rangle$. Dessa forma, se espera que $k = 1$ e $\vec{F_A} = \langle 0; 2 \rangle$. Para fazer as substituições, vamos fazer um dicionário com os valores conhecidos.

In [16]:
vals = {
    rxa: 0, rya: 2,
    rxb: 4, ryb: 2,
    cx: 0, cy: -30,
    ix: 0, iy: 1,
    mr: -50
    }

In [17]:
vals.update({
    phi: exp_phi.subs(vals),
    psi: exp_psi.subs(vals)
    })  # determina os valores de psi e phi

Antes de determinar os valores de $A_x$ e $A_y$, vamos determinar se o denominador das expressões é diferente de $0$.

In [18]:
vals[psi] - vals[phi]

-4

In [19]:
vals.update({ax: expAxs.subs(vals), ay: expAys.subs(vals)})
print(vals[ax], vals[ay])

0 -35/2


In [20]:
expk.subs(vals)

-25/2

## Considerações finais

Tendo em vista o desenvolvimento mostrado, conclui-se que a intensidade do vetor de direção restrita $\vec{F_B}$ é equivalente à expressão:

In [21]:
expk

(A_x*R_yA - A_y*R_xA + M_R)/\phi

Onde

$\phi = |\vec{R_B} \times \hat{i}|$

e

$\psi = |\vec{R_A} \times \hat{i}|$

In [22]:
exp_phi

R_xB*i_y - R_yB*i_x

In [23]:
exp_psi

R_xA*i_y - R_yA*i_x

Enquanto que $A_x$ e $A_y$, que compõe $\vec{F_A}$, são determinados pelas respectivas fórmulas:

In [24]:
expAxs

(C_x*R_xA*i_y - C_x*\phi - C_y*R_xA*i_x + M_R*i_x)/(-\phi + \psi)

In [25]:
expAys

(C_x*R_yA*i_y - C_y*R_yA*i_x - C_y*\phi + M_R*i_y)/(-\phi + \psi)